# 기상청 날씨 데이터 가져오기

In [ ]:
import requests
import pandas as pd
import json

In [ ]:
## 기본 url 생성
def url_prod(startDt, endDt, viewRow=10, location=159):
    uKey='VFYXI%2F3A5kIk7QVsO1epAlZSb6ZtCO9N5VC%2FMF3pW6oZ7%2BsFrvH1Y9y0UfmVzTnA0k2FasoNM6t6JsY7%2BEwNEw%3D%3D'

    url='http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey=' + uKey
    url += '&numOfRows=' + str(viewRow) + '&pageNo=1&&dataType=JSON'
    url += '&dataCd=ASOS&dateCd=DAY&startDt=' + str(startDt) + '&endDt=' + str(endDt)
    url += '&stnIds=' + str(location)

    weather_html=requests.get(url)

    if weather_html.status_code != 200: exit('데이터 없음')
    weather_json=json.loads(weather_html.text)

    return weather_json


In [ ]:
startDt=input('시작일 입력:(예:20210101)')
endDt=input('종료일 입력: (예:20221231)')
location=input('조회지역: ')

loc_code_df=pd.read_csv('./관측지점코드.csv', encoding='cp949')
tmp=loc_code_df[loc_code_df['지점명']==location].index
location=loc_code_df.loc[tmp[0], '지점']

weather_json=url_prod(startDt, endDt, location)
viewRow=weather_json['response']['body']['totalCount']

weather_json=url_prod(startDt, endDt, viewRow)

items=weather_json['response']['body']['items']['item']

weather_df=pd.DataFrame(items)
weather_df.info()

In [ ]:
wdf_sub=weather_df[['tm','stnNm', 'avgTa', 'minTa','maxTa', 'sumRn', 'maxInsWs',
                    'avgWs', 'avgRhm']].copy()

wdf_sub.head()

In [ ]:
wdf_sub.isna().sum()

In [ ]:
wdf_sub[wdf_sub['sumRn']=='']

In [ ]:
wdf_sub.loc[wdf_sub['sumRn']=='','sumRn'] = 0.0
wdf_sub